In [ ]:
import pandas as pd
from psycopg2 import connect
from faker import Faker

faker = Faker(locale='en-AU')

PGCONN = "postgresql://app_user:password@db/app_db"

### Create the tables & data

In [ ]:
N_USERS = 1000
N_CARDS = 500
N_ITEMS = 50
N_PURCHASES = 2000

user_table = pd.DataFrame(
    [
        {
            "uid": i,
            "first_name": faker.first_name(),
            "last_name": faker.last_name(),
            "dob": faker.date_of_birth(),
            "street_address": faker.address(),
            "phone_number": faker.phone_number(),
            "email": faker.ascii_email(),
            
        }
        for i in range(N_USERS)
    ]
)

card_table = pd.DataFrame(
    [
        {
            "cid": i,
            "uid": faker.random_element(user_table['uid']),
            "card_number": faker.credit_card_number(),
            "card_security": faker.credit_card_security_code(),
            "card_expiry": faker.date_between(start_date='today', end_date='+5y'),
        }
        for i in range(N_CARDS)
    ]
)

def fake_item_name():
    size = faker.random_element(["Extra-Small", "Small", "Medium", "Large", "Extra-Large"])
    colour = faker.color_name()
    thing = faker.random_element([
        "Toaster", "Fridge", 
        "Chair", "Table", 
        "T-Shirt", "Shoes", 
        "Bike", "Ball",
        "DVD Movie", "8-Track",
    ])
    return f"{size} {colour} {thing}"


inventory_table = pd.DataFrame(
    [
        {
            "iid": i,
            "item_code": faker.msisdn(),
            "item_name": fake_item_name(),
            "manufacturer": faker.company(),
        }
        for i in range(N_ITEMS)
    ]
)

purchase_table = pd.DataFrame(
    [
        {
            "pid": i,
            "uid": faker.random_element(user_table['uid']),
            "cid": faker.random_element(card_table['cid']),
            "iid": faker.random_element(inventory_table['iid']),
            "transaction_date": faker.date_this_year(),
            "count": faker.random_int(min=1, max=10),
            "item_cost": faker.random_int(min=100, max=100_000)/100.,
        }
        for i in range(N_PURCHASES)
    ]
)

### Create the tables

In [ ]:
for df, table_name in [
    (purchase_table, "purchase"),
    (inventory_table, "inventory"),
    (card_table, "card"),
    (user_table, "end_user"),
]:
    df.to_sql(table_name, schema='public', con=PGCONN, if_exists='replace', index=False, method='multi')

### Create primary keys, indexes, relationships

In [ ]:
with connect(PGCONN) as conn:
    cur = conn.cursor()
    for table, col in [
        ('end_user', 'uid'),
        ('card', 'cid'),
        ('inventory', 'iid'),
        ('purchase', 'pid'),
    ]:
        cur.execute(f"ALTER TABLE {table} ADD PRIMARY KEY ({col});")
        
    for t, ft, col in [
        ("card", "end_user", "uid"),
        ("purchase", "end_user", "uid"),
        ("purchase", "card", "cid"),
        ("purchase", "inventory", "iid"),
    ]:
        cur.execute(f"ALTER TABLE {t} ADD CONSTRAINT {t}_{col}_fk FOREIGN KEY ({col}) REFERENCES {ft} ({col});")

### Inspect the result with pandas

In [ ]:
pd.read_sql("SELECT * FROM pg_tables WHERE schemaname = 'public'", con=PGCONN)

In [ ]:
pd.read_sql("SELECT * FROM pg_indexes WHERE schemaname = 'public'", con=PGCONN)

In [ ]:
pd.read_sql(
    """
    SELECT conrelid::regclass AS table_from
         , conname AS constraint_name
         , pg_get_constraintdef(c.oid) as constraint_def
    FROM   pg_constraint c
    JOIN   pg_namespace n ON n.oid = c.connamespace
    WHERE  contype IN ('f', 'p ')
    AND    n.nspname = 'public' -- your schema here
    ORDER  BY conrelid::regclass::text, contype DESC;
    """, 
    con=PGCONN
)

# NOTE
You will now need to issue 

`root:/ # kill -SIGUSR1 1`

within the postgrest container, so that postgrest can restart, refreshing it's cache of FK relationships.

If you do not do this, the REST API will not function correctly.